<a href="https://colab.research.google.com/github/nsinghal7/ping_pong_robot/blob/main/pingpong.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import importlib
import os, sys
from urllib.request import urlretrieve

if 'google.colab' in sys.modules and importlib.util.find_spec('manipulation') is None:
    urlretrieve(f"http://manipulation.csail.mit.edu/scripts/setup/setup_manipulation_colab.py",
                "setup_manipulation_colab.py")
    from setup_manipulation_colab import setup_manipulation
    setup_manipulation(manipulation_sha='master', drake_version='20201101', drake_build='nightly')

from IPython import get_ipython
running_as_notebook = get_ipython() and hasattr(get_ipython(), 'kernel')

# setup ngrok server
server_args = []
if 'google.colab' in sys.modules:
  server_args = ['--ngrok_http_tunnel']

# start a single meshcat server instance to use for remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
try:
  proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=server_args)
except e:
  print(e)

import numpy as np
from IPython.display import display, HTML
from pydrake.examples.manipulation_station import ManipulationStation

import matplotlib.pyplot as plt, mpld3
if running_as_notebook:
  mpld3.enable_notebook()

import pydrake
from pydrake.all import (
    DiagramBuilder, ConnectMeshcatVisualizer, Simulator, FindResourceOrThrow,
    Parser, MultibodyPlant, RigidTransform, LeafSystem, BasicVector,
    JacobianWrtVariable, RollPitchYaw, SignalLogger, AddTriad,
    PiecewisePolynomial, PiecewiseQuaternionSlerp, RotationMatrix, Solve,
    TrajectorySource, AddMultibodyPlantSceneGraph, PlanarSceneGraphVisualizer,
    Value, ExternallyAppliedSpatialForce_, SpatialForce_, LeafSystem_,
    ExternallyAppliedSpatialForce, SpatialForce, BasicVector_, if_then_else, Expression,
    SpatialVelocity, ConstantVectorSource, PrismaticJoint, FixedOffsetFrame,
    JointActuator, MathematicalProgram, eq
)

from pydrake.systems.scalar_conversion import TemplateSystem
from pydrake.common.cpp_param import List
from pydrake.multibody import inverse_kinematics
from pydrake.all import SnoptSolver, IpoptSolver
from pydrake.trajectories import PiecewisePolynomial
import urllib.request
from IPython.display import HTML
from collections import namedtuple
import decimal

Cloning into '/opt/manipulation'...

Already on 'master'

/sbin/ldconfig.real: /usr/local/lib/python3.6/dist-packages/ideep4py/lib/libmkldnn.so.0 is not a symbolic link

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK

ERROR: tensorflow 2.3.0 has requirement scipy==1.4.1, but you'll have scipy 1.5.3 which is incompatible.
ERROR: nbclient 0.5.1 has requirement jupyter-client>=6.1.5, but you'll have jupyter-client 5.3.5 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.






In [ ]:
def AdvanceToAndVisualize(sim, viz, time):
  viz.start_recording()
  for t in range(1, 11):
    print("simulating part %d/10" % t)
    sim.AdvanceTo(t * time / 10)
  print("animating...")
  ani = viz.get_recording_as_animation()
  display(HTML(ani.to_jshtml()))

def float_to_string(f):
  ctx = decimal.Context()
  ctx.prec = 20
  d1 = ctx.create_decimal(repr(f))
  return format(d1, 'f')

def calc_cylinder_info(mass, radius, length):
  ix = mass * (3*radius**2 + length**2) / 12
  iz = mass * radius**2 / 2
  return {"m": mass, "r": radius, "l": length, "ix": ix, "iz": iz}

def calc_physical_values():
  head = calc_cylinder_info(.042, .08, .016)
  handle = calc_cylinder_info(.008, .013, .1)
  head["zoffset"] = head["r"] + handle["l"] / 2
  ball = {
      "m": .0027,
      "r": .02
  }
  ball["i"] = 2.*ball["m"]*ball["r"]**2 / 3.

  table = {"x": 0, "y": 0, "z": 0, "wx": 1.525, "wy": 1.37, "wz": .04}
  wall = {"wx": 1.525, "wy": .04, "wz": 1.37}
  wall["yoffset"] = table["wy"] / 2 + wall["wy"] / 2
  wall["zoffset"] = table["wz"] / 2 + wall["wz"] / 2
  wall["x"] = table["x"]
  wall["y"] = table["y"] + wall["yoffset"]
  wall["z"] = table["z"] + wall["zoffset"]
  
  return {"head": head, "handle": handle, "ball": ball, "table": table, "wall": wall, "pi": np.pi, "hpi": np.pi/2}

def access(pv, expr):
  lst = expr.split(".")
  for n in lst:
    pv = pv[n]
  return pv

def load_urdf(path):
  % cd /content
  %shell [ ! -d pingpong ] && git clone https://github.com/nsinghal7/ping_pong_robot.git pingpong || echo "pingpong already exists"
  %shell cd pingpong && git pull
  % cd /content
  with open(path) as f:
    orig = f.read()
  
  ans = ""
  pv = calc_physical_values()
  while True:
    next = orig.find("${")
    if next == -1:
      ans += orig
      break
    else:
      ans += orig[:next]
      orig = orig[next + 2:]
      end = orig.find("}")
      assert(end != -1)
      assert(end < 20) # let's pretend this makes it safe to use eval
      ans += float_to_string(access(pv, orig[:end]))
      orig = orig[end+1:]
  return ans

In [ ]:
# Represents axis-aligned rectangles that can be contacted only on one side
# low is corner with lowest coord values, high is the opposite
# in the normal axis, if sign (-1, +1) is the direction of the normal force
Surface = namedtuple('Surface', ['low', 'high', 'sign'])

# Calculates and applies bouncing forces and drag/magnus forces
@TemplateSystem.define("ForceCalculator_")
def ForceCalculator_(T):
  class Impl(LeafSystem_[T]):
    def _construct(self, ball_radius, paddle_radius, paddle_thickness, surfaces, plant, converter=None):
      LeafSystem_[T].__init__(self, converter=converter)
      self.ball_radius = ball_radius
      self.paddle_radius = paddle_radius
      self.paddle_thickness = paddle_thickness
      self.surfaces = surfaces
      self.plant = plant
      self.on = False
      
      self.mbp_state_input_port = self.DeclareVectorInputPort(
          "mbp_state_input_port", BasicVector_[T](31))
      forces_cls = Value[List[ExternallyAppliedSpatialForce_[T]]]
      self.DeclareAbstractOutputPort(
          "spatial_forces_vector",
          lambda: forces_cls(),
          self.DoCalcForceOutput)
      
    def _construct_copy(self, other, converter=None):
      Impl._construct(self, other.ball_radius, other.paddle_radius,
                      other.paddle_thickness, other.surfaces,
                      other.plant, converter=converter)
    
    def DoCalcForceOutput(self, context, output_data):
      if T != float:
        print("ignoring symbolic access to bounce")
        output_data.set_value([])
        return
      
      state = np.array(self.mbp_state_input_port.Eval(context))
      ball_ori = state[0:4]
      ball_pos = state[4:7]
      ball_rotv = state[16:19]
      ball_linv = state[19:22]

      forces = []

      for surface in self.surfaces:
        if surface.low[0] == surface.high[0]:
          normal = [surface.sign, 0, 0]
        elif surface.low[1] == surface.high[1]:
          normal = [0, surface.sign, 0]
        elif surface.low[2] == surface.high[2]:
          normal = [0, 0, surface.sign]
        else:
          raise ValueError("no clear normal direction!")
        normal = np.array(normal)
        
        low_offset = ball_pos - surface.low
        norm_dist = low_offset.dot(normal)
        planar_low_offset = low_offset - norm_dist * normal
        assert abs(planar_low_offset.dot(normal)) < 1e-8

        if np.any((planar_low_offset < 0) | (planar_low_offset > surface.high - surface.low)) or norm_dist - self.ball_radius > 0 or norm_dist < 0:
          # ball isn't touching plane. np.any covers the case of offset from the edges of the rectangle,
          # norm - radius > 0 covers ball in front of rectangle, norm < 0 covers ball behind rectangle.
          # we allow ball to overlap with rectangle from behind in case we want to make a "net" with two close together surfaces back to back
          # TODO: handle edge of ball hitting the edge of the surface. Not important for this demo probably...
          continue
        else:
          norm_linv = ball_linv.dot(normal)
          dx = norm_dist - self.ball_radius
          self.CalcForce(dx, norm_linv, normal, forces)
      
      # Paddle contact
      plant_context = self.plant.CreateDefaultContext()
      plant_context.SetDiscreteState(state)
      paddle_pose = self.plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("paddle_head"))
      paddle_vel = self.plant.EvalBodySpatialVelocityInWorld(plant_context, plant.GetBodyByName("paddle_head"))
      ball_pose = self.plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("ball"))
      ball_vel = self.plant.EvalBodySpatialVelocityInWorld(plant_context, plant.GetBodyByName("ball"))

      X_Ball = RigidTransform(ball_pose)
      X_Paddle = RigidTransform(paddle_pose)
      X_PaddleBall = X_Paddle.inverse().multiply(X_Ball)
      center_offset = X_PaddleBall.translation()
      dx = abs(center_offset[2]) - self.ball_radius - self.paddle_thickness / 2
      if np.linalg.norm(center_offset[:2]) <= self.paddle_radius and dx <= 0:
        # we have a collision with the paddle!!
        # TODO: include collisions with edge of paddle
        side_sign = 1 if center_offset[2] > 0 else -1
        paddle_offset = np.array([*center_offset[:2], side_sign*self.paddle_thickness])
        paddle_contactpt_vel = paddle_vel.Shift(paddle_offset) # only care about translational part since we contact on this point
        V_StaticpaddleBall = ball_vel.Rotate(X_Paddle.rotation().inverse())
        linv_rel = V_StaticpaddleBall.translational() - paddle_contactpt_vel.translational()
        rotv_rel = V_StaticpaddleBall.rotational() # to be used for friction
        force_dir = X_Paddle.rotation().multiply(np.array([0, 0, side_sign]))

        self.CalcForce(dx, linv_rel[2]*side_sign, force_dir, forces)
        self.on = True
      else:
        if self.on: print()
        self.on = False
      

      # TODO: add forces on paddle
      exforce = ExternallyAppliedSpatialForce_[T]()
      exforce.body_index = self.plant.GetBodyByName("paddle_handle").index()
      exforce.p_BoBq_B = np.array([0, 0, .1092])
      exforce.F_Bq_W = SpatialForce_[T](tau=[0, 0, 0], f=[0, 0, 9.81*.05])
      forces.append(exforce)


      output_data.set_value(forces)
    
    def CalcForce(self, dx, norm_linv, normal, forces):
      normal_force = -1000*dx - 0.18 * (-250*dx)* norm_linv # made up parameters, but they seem reasonable
      # TODO add friction (and update p_BoBq_B!!!)

      exforce = ExternallyAppliedSpatialForce_[T]()
      ball_body = self.plant.GetBodyByName("ball")
      exforce.body_index = ball_body.index()
      exforce.p_BoBq_B = np.zeros(3) # TODO: this is fine for normal force, but won't work for friction if we want torque
      exforce.F_Bq_W = SpatialForce_[T](tau=[0.,0.,0.], f=normal*normal_force)
      forces.append(exforce)
  
  return Impl




In [ ]:
@TemplateSystem.define("PingPongPlanner_")
def PingPongPlanner_(T):
  class Impl(LeafSystem_[T]):
    def _construct(self, plant, controller_plant, pv, velocify, converter=None):
      LeafSystem_[T].__init__(self, converter=converter)
      self.plant = plant
      self.controller_plant = controller_plant
      self.pv = pv
      self.velocify = velocify
      self.ts = 0.1 # how often to recompute goal
      self.last_update_ts = -1
      self.q_goal = np.zeros(9)
      self.v_goal = np.zeros(9)
      self.damping = 0.85
      self.g = -9.81
      self.behind_table = 0.1
      self.max_return_height = 1.0
      self.return_vy = 3
      self.net_height = 0.2
      
      self.mbp_state_input_port = self.DeclareVectorInputPort(
          "mbp_state_input_port", BasicVector_[T](31))
      self.DeclareVectorOutputPort(
          "iiwa_desired_state", BasicVector_[T](18), self.DoCalcDesPos)
    
    def _construct_copy(self, other, converter=None):
      Impl._construct(self, other.plant, other.controller_plant, other.pv, other.velocify,
                      converter=converter)
  
    def projectile(self, x,y,z,vx,vy,vz,dt):
      return x + vx*dt, y + vy*dt, z + vz*dt + self.g*dt*dt/2, vz + self.g*dt
  
    def z_time_diff(self, dz, vz):
      return (-vz - np.sqrt(vz**2 - 2*dz*self.g + 1e-15)) / self.g
  
    def bounce_until_y(self, x,y,z,vx,vy,vz, goal_y, reaches_z):
      time_to_goal = (goal_y - y) / vy
      while z + vz * time_to_goal + self.g*time_to_goal**2/2 <= reaches_z:
        # we hit the table before hitting the goal
        dt = self.z_time_diff(z - reaches_z, vz)
        x,y,z,vz = self.projectile(x,y,z,vx,vy,vz, dt)
        vz*=-self.damping
        time_to_goal -= dt
        if dt < 1e-6:
          # we are basically rolling, so roll to the goal
          x += vx * time_to_goal
          y += vy * time_to_goal
          z = reaches_z
          dz = 0
          time_to_goal = 0
          break
        
      # we continue until the goal
      dt = time_to_goal
      x,y,z,vz = self.projectile(x,y,z,vx,vy,vz,dt)
      return x,y,z,vz
      
    def predict_next_hit_ball_state(self, ball_pos, ball_linv):
      # must handle events: hits table, hits wall, reaches hittable zone
      # hittable zone: any x (if it's unreachable then no hope), y is past table
      #                z is below 1m
      # For now, just hit ball back to center at 0.5m above table on return
      x, y, z = ball_pos
      vx, vy, vz = ball_linv
      table_z = self.pv["table"]["z"] + self.pv["table"]["wz"] / 2
      r = self.pv["ball"]["r"]
      if vy > 0:
        wall_y = self.pv["wall"]["y"] - self.pv["wall"]["wy"]/2
        x,y,z,vz = self.bounce_until_y(x,y,z,vx,vy,vz, wall_y, table_z + r)
        vy*=-self.damping
      
      border_y = -self.pv["table"]["wy"]/2 - self.behind_table
      x,y,z,vz = self.bounce_until_y(x,y,z,vx,vy,vz, border_y, table_z + r)

      hittable_height = self.max_return_height
      if z > hittable_height: # we can't always reach a high ball
        dt = self.z_time_diff(z - r - hittable_height, vz)
        x,y,z,vz = self.projectile(x,y,z,vx,vy,vz,dt)
      
      # ball must now be in the hittable zone
      return np.array([x,y,z]), np.array([vx,vy,vz])

    def calc_vout(self, ball_pos, ball_vin):
      # calc output velocities that will result in returning
      # with x=0, y=ybarrier, vy=return_vy, in this case assuming damping affects all velocity components
      wall_y = self.pv["wall"]["y"] - self.pv["wall"]["wy"]/2
      border_y = -self.pv["table"]["wy"]/2 - self.behind_table
      ady = self.damping*(wall_y - ball_pos[1]) + (wall_y - border_y)
      dt = ady/self.return_vy
      vyo = self.return_vy / self.damping
      vxo = -ball_pos[0] / dt
      leftover = self.damping**2*ball_vin.dot(ball_vin) - vyo**2 - vxo**2
      while leftover < -1e-12:
        vyo*=.9
        vxo*=.9
        leftover = self.damping**2*ball_vin.dot(ball_vin) - vyo**2 - vxo**2
      vzo = np.sqrt(leftover + 1e-8)

      vo = np.array([vxo, vyo, vzo])
      return vo

    def calc_head_z(self, ball_vin, ball_vout):
      vin = ball_vin / np.linalg.norm(ball_vin)
      vout = ball_vout / np.linalg.norm(ball_vout)
      head_z = (vout - vin)
      head_z_norm = np.linalg.norm(head_z)
      if head_z_norm < 1e-8:
        print("Got low average velocity for head z: ", head_z)
        return np.array([0, 1, 0])
      return head_z / head_z_norm
    
    def calc_head_z_head_v(self, ball_pos, ball_vin):
      wall_y = self.pv["wall"]["y"] - self.pv["wall"]["wy"]/2
      border_y = -self.pv["table"]["wy"]/2 - self.behind_table
      ady = self.damping*(wall_y - ball_pos[1]) + (wall_y - border_y)
      dt = ady/self.return_vy
      vyo = self.return_vy / self.damping
      vxo = -ball_pos[0] / dt
      dt1 = (wall_y - ball_pos[1]) / vyo
      vzo = (self.net_height + self.pv["ball"]["r"] - ball_pos[2] - self.g*dt1**2/2)/dt1
      vo = np.array([vxo, vyo, vzo])
      prog = MathematicalProgram()
      head_z = prog.NewContinuousVariables(3, "head_z")
      head_v = prog.NewContinuousVariables(3, "head_v")
      prog.AddConstraint(head_z.dot(head_z) == 1)
      vrel = ball_vin - head_v
      vrel_hnorm = vrel.dot(head_z)
      vrel_out = vrel - 2*vrel_hnorm*head_z
      prog.AddConstraint(eq(vrel_out + head_v, vo))
      prog.AddLinearConstraint(head_z[1] >= 0)
      head_z_guess = self.calc_head_z(ball_vin, vo)
      prog.SetInitialGuess(head_z, head_z_guess)
      prog.SetInitialGuess(head_v, np.zeros(3))
      result = Solve(prog)
      if not result.is_success():
        print("failed to solve for vout falling back to basic")
        return head_z_guess, np.zeros(3)
      print(vo, result.GetSolution(head_v))
      return result.GetSolution(head_z), result.GetSolution(head_v)
      
      
      # prog = MathematicalProgram()
      # ts = prog.NewContinuousVariables(4, "t")
      # vs = prog.NewContinuousVariables(3, 3, "v")
      # ps = prog.NewContinuousVariables(4, 3, "ps")
      # prog.AddConstraint(eq(ps[1:, 0:2], ps[0:3, 0:2] + vs[0:3, 0:2]*ts[0:3, None]))
      # prog.AddConstraint(eq(ps[1:, 2], ps[0:3, 2] + vs[0:3, 2]*ts[0:3]))
      # prog.AddConstraint(eq(vs[1:, 0], vs[[0, 0, 0], 0]))
      # prog.AddConstraint(vs[1, 1] == vs[0, 1])
      # prog.AddConstraint(vs[2, 1] == -self.damping*vs[0, 1])
      # prog.AddConstraint(vs[1, 2] == vs[0, 2] + self.g*ts[0]
      # prog.AddConstraint(vs[2, 2] == -self.damping*(vs[1, 2] + self.g*ts[1]))
      # prog.AddConstraint(ps[3, 0] == 0)
      # prog.AddLinearConstraint(ps[1, 1] == wall_y)
      # prog.AddLinearConstraint(ps[3, 1], border_y - .05, border_y + .05)
      # prog.AddLinearConstraint(ps[3, 2], zmin, self.max_return_height)
      # prog.AddLinearConstraint(ps[1, 2] > self.net_height)
      # prog.AddLinearConstraint(ps[2, 2] == zmin)
      # prog.AddLinearConstraint(eq(ps[0, :], ball_pos))
      # prog.AddCost((vs[0, 1] - self.return_vy)**2)
      # prog.SetInitialGuess(ts, np.array([.37, 0.5, 0.75]))
      # prog.SetInitialGuess(ps, np.array([ball_pos, [0, wall_y, zmin], [0, wall_y/2, zmin], [0, border_y, zmin]]))
      # result = Solve(prog)
      # if not result.is_success():
      #   print("failed to solve for vout. falling back to basic")
      #   vout = self.calc_vout(ball_pos, ball_vin)
      #   head_z = self.calc_head_z(ball_vin, ball_vout)
      #   head_v = np.zeros(3)
      #   return vout, head_z, head_v
      


    def update_q_goal(self, context):
      state = np.array(self.mbp_state_input_port.Eval(context))
      ball_ori = state[0:4]
      ball_pos = state[4:7]
      iiwa_pos = state[7:16]
      ball_rotv = state[16:19]
      ball_linv = state[19:22]
      iiwa_vel = state[22:31]

      hit_pos, hit_vel = self.predict_next_hit_ball_state(ball_pos, ball_linv)
      if not self.velocify:
        vout = self.calc_vout(hit_pos, hit_vel)
        head_z = self.calc_head_z(hit_vel, vout)
      else:
        head_z, self.v_goal = self.calc_head_z_head_v(hit_pos, hit_vel)
        # print(head_z, self.v_goal)

      p_des = hit_pos - head_z*self.pv["head"]["l"]/2
      arm = 1
      arm_des_w = p_des + head_z*arm
      arm_des_h = [0, 0, arm]

      ik = inverse_kinematics.InverseKinematics(self.controller_plant)
      q_variables = ik.q()
      prog = ik.prog()
      world_frame = self.controller_plant.world_frame()
      head_frame = self.controller_plant.GetFrameByName("paddle_head")
      dp = np.array([.004, .004, .004])
      dtheta = np.array([.007, .007, 100])*arm + dp
      ik.AddPositionConstraint(frameA=world_frame,
                               frameB=head_frame,
                               p_BQ=np.zeros(3),
                               p_AQ_lower=p_des - dp,
                               p_AQ_upper=p_des + dp)
      # replace orientation constraint with constraint on point arm dist in front of paddle
      # this way, yawing paddle doesn't matter, nor does the side of the paddle
      ik.AddPositionConstraint(frameA=head_frame,
                               frameB=world_frame,
                               p_BQ=arm_des_w,
                               p_AQ_lower=arm_des_h - dtheta,
                               p_AQ_upper=arm_des_h + dtheta)
      if self.velocify:
        prog.AddLinearConstraint(q_variables[-1] == 0) # TODO: remove this and replace with a constraint requiring a useful jacobian??
      prog.AddQuadraticErrorCost(np.eye(9), iiwa_pos, q_variables)
      prog.SetInitialGuessForAllVariables(iiwa_pos)
      result = Solve(prog)
      if not result.is_success():
        print("failed to solve ik: the weird version")
        return
      self.q_goal = result.GetSolution(q_variables)
    
    def ball_is_close(self, context):
      state = np.array(self.mbp_state_input_port.Eval(context))
      ball_ori = state[0:4]
      ball_pos = state[4:7]
      iiwa_pos = state[7:16]
      ball_rotv = state[16:19]
      ball_linv = state[19:22]
      iiwa_vel = state[22:31]
      plant_context = self.controller_plant.CreateDefaultContext()
      plant_context.SetDiscreteState(np.concatenate([iiwa_pos, iiwa_vel]))
      head_pose = self.controller_plant.EvalBodyPoseInWorld(plant_context, self.controller_plant.GetBodyByName("paddle_head"))
      return np.linalg.norm(head_pose.translation() - ball_pos) < 0.2*-ball_linv[1], np.linalg.norm(head_pose.translation() - ball_pos) < 0.13*-ball_linv[1]


    def DoCalcDesPos(self, context, output_data):
      if T != float:
        print("ignoring symbolic access to ping pong planner")
        output_data.set_value(np.zeros(18))
        return
      
      now = context.get_time()
      close_threshs = self.ball_is_close(context)
      is_close = close_threshs[0] and self.velocify
      if (self.last_update_ts > now or now - self.last_update_ts > self.ts) and not is_close:
        self.update_q_goal(context)
        self.last_update_ts = now
      elif is_close:
        state = np.array(self.mbp_state_input_port.Eval(context))
        iiwa_pos = state[7:16]
        iiwa_vel = state[22:31]
        plant_context = self.controller_plant.CreateDefaultContext()
        plant_context.SetDiscreteState(np.concatenate([iiwa_pos, iiwa_vel]))
        world_frame = self.controller_plant.world_frame()
        J_G = self.controller_plant.CalcJacobianSpatialVelocity(plant_context,
                                                                JacobianWrtVariable.kQDot,
                                                                self.controller_plant.GetFrameByName("paddle_head"),
                                                                [0, 0, 0], world_frame, world_frame)
        J_G = J_G[3:6, :]
        prog = MathematicalProgram()
        v = prog.NewContinuousVariables(9, 'v')
        error = J_G.dot(v) - self.v_goal
        prog.AddCost(error.T.dot(np.diag([100, 30, 40])).dot(error))
        v_max = np.ones(9)*.3
        v_max[:2] = .01
        prog.AddBoundingBoxConstraint(-v_max, v_max, v)
        prog.SetInitialGuessForAllVariables(np.zeros(9))
        result = Solve(prog)
        if not result.is_success():
          print("failed convert v J_G")
          v = np.zeros(9)
        else:
          v = result.GetSolution(v)
        
        if not close_threshs[1]:
            # first half of ball's approach, we should back away
            v *= -1
        output_data.set_value(np.concatenate([self.q_goal, 5*(v - iiwa_vel) + iiwa_vel])) # hack velocity to overcome low kd
        return
      
      output_data.set_value(np.concatenate([self.q_goal, np.zeros(9)]))
  
  return Impl
      

In [ ]:
def setup_plant(plant, pv, paddle_only):
  parser = Parser(plant)
  if not paddle_only:
    parser.AddModelFromString(load_urdf("/content/pingpong/ball_and_table.urdf"),
                              "urdf",
                              "pingpong")
  parser.AddModelFromString(load_urdf("/content/pingpong/paddle.urdf"),
                            "urdf",
                            "paddle")
  iiwa_model_instance_index = parser.AddModelFromFile("/content/pingpong/iiwa.sdf")
  plant.WeldFrames(
      A=plant.world_frame(),
      B=plant.GetFrameByName("iiwa_base_x"),
      X_AB=RigidTransform(RollPitchYaw(0, 0, np.pi/2), np.array([0, -pv["table"]["wy"]/2 - 0.7, -.3]))
  )
  plant.WeldFrames(
      A=plant.GetFrameByName("iiwa_link_7"),
      B=plant.GetFrameByName("paddle_handle"),
      X_AB=RigidTransform(RollPitchYaw(-np.pi/2, np.pi/2, 0), np.array([0, pv["handle"]["l"]/2, pv["handle"]["l"]/2]))
  )
  return iiwa_model_instance_index

def setup_ping_pong(velocify, ts=1e-4, penetration_allowance=1e-4):
  builder = DiagramBuilder()
  plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=ts)
  pv = calc_physical_values()
  iiwa_model_instance_index = setup_plant(plant, pv, False)
  plant.Finalize()
  plant.set_penetration_allowance(penetration_allowance)

  t = pv["table"]
  w = pv["wall"]
  force_calc = builder.AddSystem(ForceCalculator_[float](
      pv["ball"]["r"], pv["head"]["r"], pv["head"]["l"],
      [Surface(np.array([t["x"] - t["wx"]/2, t["y"] - t["wy"]/2, t["z"] + t["wz"]/2]),
               np.array([t["x"] + t["wx"]/2, t["y"] + t["wy"]/2, t["z"] + t["wz"]/2]),
               1),
       Surface(np.array([w["x"] - w["wx"]/2, w["y"] - w["wy"]/2, w["z"] - w["wz"]/2]),
               np.array([w["x"] + w["wx"]/2, w["y"] - w["wy"]/2, w["z"] + w["wz"]/2]),
               -1)
      ],
      plant))

  meshcat = ConnectMeshcatVisualizer(builder,
                                     scene_graph,
                                     output_port=scene_graph.get_pose_bundle_output_port(),
                                     delete_prefix_on_load=False,
                                     frames_to_draw={"pingpong": {"ball", "table", "wall"}, "paddle": {"paddle_head", "paddle_handle"}},
                                     zmq_url=zmq_url)

  builder.Connect(plant.get_state_output_port(), force_calc.get_input_port(0))
  builder.Connect(force_calc.get_output_port(0), plant.get_applied_spatial_force_input_port())

  # iiwa controller setup
  controller_plant = MultibodyPlant(ts);
  controller_iiwa = setup_plant(controller_plant, pv, True)
  controller_plant.Finalize()

  iiwa_controller = builder.AddSystem(
      pydrake.systems.controllers.InverseDynamicsController(
          controller_plant,
          kp=[100]*9,
          ki=[1]*9,
          kd=[20]*9,
          has_reference_acceleration=False))
  iiwa_controller.set_name("iiwa_controller")
  builder.Connect(plant.get_state_output_port(iiwa_model_instance_index), iiwa_controller.get_input_port_estimated_state())
  builder.Connect(iiwa_controller.get_output_port_control(), plant.get_actuation_input_port(iiwa_model_instance_index))

  # desired_state_from_position = builder.AddSystem(
  #     pydrake.systems.primitives.StateInterpolatorWithDiscreteDerivative(
  #         9, ts, suppress_initial_transient=True))
  # desired_state_from_position.set_name("desired_state_from_position")
  # builder.Connect(desired_state_from_position.get_output_port(),
  #                 iiwa_controller.get_input_port_desired_state())

  planner = builder.AddSystem(PingPongPlanner_[float](plant, controller_plant, pv, velocify))
  builder.Connect(plant.get_state_output_port(), planner.get_input_port())
  builder.Connect(planner.get_output_port(), iiwa_controller.get_input_port_desired_state())

  diagram = builder.Build()
  sim = Simulator(diagram)
  simcon = sim.get_mutable_context()
  plant_simcon = plant.GetMyMutableContextFromRoot(simcon)
  plant_simcon.SetDiscreteState( [
                            1, 0, 0, 0, # ball orientation
                            0, -.5, 1, # ball position

                            0, 0, 0, .1, 0, -1.2, 0, 1.6, 0, # iiwa joint position

                            0, 0, 0, # ball rotational vel
                            .1, 4, 0, # ball linear vel

                            0, 0, 0, 0, 0, 0, 0, 0, 0 # iiwa joint vel
                          ])

  return sim, simcon, plant, force_calc, meshcat

In [ ]:
# meshcat.reset_recording()
sim, simcon, plant, force_calc, meshcat = setup_ping_pong(True)
meshcat.start_recording()
sim.AdvanceTo(10)
meshcat.stop_recording()
meshcat.publish_recording()

/content
pingpong already exists
Already up to date.
/content
/content
pingpong already exists
Already up to date.
/content
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6000...
You can open the visualizer by visiting the following URL:
http://582501fccf1d.ngrok.io/static/
Connected to meshcat-server.
/content
pingpong already exists
Already up to date.
/content
ignoring symbolic access to bounce
ignoring symbolic access to ping pong planner
[-0.08037539  3.52941176  0.90484947] [-1.75951114e-03  6.74375017e-02 -8.83146753e-05]
[-0.08037539  3.52941176  0.90489036] [-1.76033298e-03  6.74689261e-02 -8.60801787e-05]
failed to solve ik: the weird version
[-0.08037539  3.52941176  0.90493122] [-1.76115390e-03  6.75003149e-02 -8.38456079e-05]
[-0.07750652  3.52941176  0.96792833] [ 0.00113057 -0.04528007  0.00091358]
[-0.07750652  3.52941176  0.96795642] [ 0.00112966 -0.04524349  0.0009115 ]
[25.24180227  3.52941176  2.47510537] [12.63437927  1.92081969  0.55465541]
failed to solv

KeyboardInterrupt: ignored